In [1]:
# импортируем необходимые библиотеки и классы
from sklearn.base import BaseEstimator, TransformerMixin
from collections import defaultdict
import pandas as pd
import numpy as np
# загружаем данные
data = pd.read_csv('Data/Rare_categories.csv', sep=';')
# выводим наблюдения
data.head()

,TARGET,GEN_INDUSTRY,GEN_TITLE,ORG_TP_STATE,ORG_TP_FCAPITAL,JOB_DIR
0,0,Торговля,Рабочий,Частная компания,Без участия,Вспомогательный техперсонал
1,0,Торговля,Рабочий,Индивидуальный предприниматель,Без участия,Участие в основ. деятельности
2,0,Информационные технологии,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности
3,0,Образование,Руководитель среднего звена,Государственная комп./учреж.,Без участия,Участие в основ. деятельности
4,0,Государственная служба,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности


In [2]:
# создаем список категориальных переменных
cat_cols = data.dtypes[data.dtypes == 'object'].index.tolist()
# смотрим частоты по категориальным переменным
for col in cat_cols:
    print(data[col].value_counts(dropna=False))
    print('')

Торговля                                     2385
Другие сферы                                 1709
NaN                                          1367
Металлургия/Промышленность/Машиностроение    1356
Государственная служба                       1286
Здравоохранение                              1177
Образование                                   998
Транспорт                                     787
Сельское хозяйство                            702
Строительство                                 574
Коммунальное хоз-во/Дорожные службы           533
Ресторанный бизнес/Общественное питание       408
Наука                                         403
Нефтегазовая промышленность                   225
Сборочные производства                        172
Банк/Финансы                                  169
Энергетика                                    145
Развлечения/Искусство                         141
ЧОП/Детективная д-ть                          136
Информационные услуги                         108


In [3]:
# записываем указанные категории переменной
# JOB_DIR в отдельную категорию OTHER
lst = ['Реклама и маркетинг', 'Юридическая служба']
data.loc[data['JOB_DIR'].isin(lst), 'JOB_DIR'] = 'OTHER'
# смотрим частоты
data['JOB_DIR'].value_counts(dropna=False)

Участие в основ. деятельности    11452
NaN                               1367
Вспомогательный техперсонал       1025
Бухгалтерия, финансы, планир.      481
Адм-хоз. и трансп. службы          279
Снабжение и сбыт                   217
Служба безопасности                164
Кадровая служба и секретариат      101
Пр-техн. обесп. и телеком.          75
OTHER                               62
Name: JOB_DIR, dtype: int64

In [4]:
# пишем класс, укрупняющий категории по порогу, категории
# с относительными частотами меньше порога запишем в
# отдельную категорию OTHER

class RareGrouper(BaseEstimator, TransformerMixin): 
    """
    Параметры:
    threshold: int, значение по умолчанию 0.01
        Минимально допустимая относительная частота, 
        при которой замены не происходит.
    """    
    def __init__(self, threshold=0.01):   
        self.d = defaultdict(list)
        self.threshold = threshold
        
    def fit(self, X, y=None):
        # для каждой переменной вычисляем относительные
        # частоты категорий и получаем словарь вида
        # defaultdict(
        #     list, {'var1': ['категория выше порога', 
        #                     'категория выше порога'],
        #            'var2': ['категория выше порога', 
        #                     'категория выше порога']})
        n_obs = len(X)
        for col in X.columns:
            rel_freq = X[col].value_counts(dropna=False) / n_obs
            self.d[col] = rel_freq[rel_freq >= self.threshold].index
        return self

    def transform(self, X):
        X = X.copy()
        # для каждой переменной категории (строковые значения), которых нет
        # в соответствующем списке словаря (их частоты ниже порога), 
        # относим к категории Other
        for col in X.columns:
            X[col] = np.where(X[col].isin(self.d[col]), X[col], 'Other')
        return X

In [5]:
# применяем класс, для переменной JOB_DIR категории 
# менее 200 наблюдений запишем в категорию OTHER, 
# порог равен 200/15223 = 0.013
raregrouper = RareGrouper(threshold=0.013)
raregrouper.fit(data[['JOB_DIR']])
data['JOB_DIR'] = raregrouper.transform(data[['JOB_DIR']])
# смотрим частоты категорий 
# переменной GEN_TITLE
data['JOB_DIR'].value_counts(dropna=False)

Участие в основ. деятельности    11452
NaN                               1367
Вспомогательный техперсонал       1025
Бухгалтерия, финансы, планир.      481
Other                              402
Адм-хоз. и трансп. службы          279
Снабжение и сбыт                   217
Name: JOB_DIR, dtype: int64

In [6]:
# укрупняем категории с помощью CHAID
from CHAID import Tree
# задаем название предиктора
independent_variable = 'GEN_TITLE'
# задаем название зависимой переменной
dep_variable = 'TARGET'
# создаем словарь, где ключом будет название 
# предиктора, а значением - тип переменной
dct = {independent_variable: 'nominal'}
# строим дерево CHAID и выводим его
tree = Tree.from_pandas_df(data, dct, dep_variable, max_depth=1)
tree.print_tree()

([], {0: 13411.0, 1: 1812.0}, (GEN_TITLE, p=9.43237916410679e-29, score=133.51911498532897, groups=[['<missing>'], ['Индивидуальный предприниматель', 'Руководитель высшего звена', 'Руководитель низшего звена', 'Военнослужащий по контракту', 'Партнер'], ['Служащий', 'Высококвалифиц. специалист', 'Работник сферы услуг', 'Рабочий'], ['Руководитель среднего звена', 'Специалист', 'Другое']]), dof=3))
|-- (['<missing>'], {0: 1317.0, 1: 50.0}, <Invalid Chaid Split> - the max depth has been reached)
|-- (['Индивидуальный предприниматель', 'Руководитель высшего звена', 'Руководитель низшего звена', 'Военнослужащий по контракту', 'Партнер'], {0: 731.0, 1: 150.0}, <Invalid Chaid Split> - the max depth has been reached)
|-- (['Служащий', 'Высококвалифиц. специалист', 'Работник сферы услуг', 'Рабочий'], {0: 4379.0, 1: 712.0}, <Invalid Chaid Split> - the max depth has been reached)
+-- (['Руководитель среднего звена', 'Специалист', 'Другое'], {0: 6984.0, 1: 900.0}, <Invalid Chaid Split> - the max de